In [17]:
import sqlite3
import os
import sys
import pandas as pd

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from glob import glob
from sqlite3 import Error


def create_pandas_DataFrame_from_string(source_str, sep=';'):
    """reference: https://stackoverflow.com/a/22605281
    
    Args:
        source_str (:obj:`str`): SQLite Table to land `source_pandas_df` in.
            * Example:
                col1;col2;col3
                1;4.4;99
                2;4.5;200
                3;4.7;65
                4;3.2;140
                
        sep (:obj:`str`): Seperator used to denote column boundaries in `source_str`.

    Returns:
        source_pandas_df (:obj:`pd.core.frame.DataFrame`): Pandas DataFrame created during method call.
    """

    TESTDATA = StringIO(source_str)
    source_pandas_df = pd.read_csv(TESTDATA, sep=sep)
    return source_pandas_df

def create_SQLite_connection(db):
    """ create a database connection to a SQLite database.
    """
    if type(db) != str:
        raise TypeError('db parameter must be str.')
    try:
        conn = sqlite3.connect(db)
    except Error as e:
        print(e)
    return conn

def push_pandas_DataFrame_to_SQLite_table(conn, source_pandas_df, destination_sqlite_table, index=False, if_exists='replace'):
    """SQLite method used to push a pandas DataFrame to a SQLite table.

    Args:
        conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
        source_pandas_df (:obj:`pd.core.frame.DataFrame`): Pandas DataFrame to push to SQLite table.
        destination_sqlite_table (:obj:`str`): SQLite Table to land `source_pandas_df` in.
    """
    if type(conn) != sqlite3.Connection:
        raise TypeError('conn parameter must be type sqlite3.Connection.')    

    if type(source_pandas_df) != pd.core.frame.DataFrame  :
        raise TypeError('source_pandas_df parameter must be pd.core.frame.DataFrame.')
        
    if type(destination_sqlite_table) != str:
        raise TypeError('destination_sqlite_table parameter must be str.')
        
    if type(index) != bool:
        raise TypeError('index parameter must be boolean.')
        
    if type(if_exists) != str:
        raise TypeError('if_exists parameter must be str.')
        
    if if_exists not in ('fail', 'replace', 'append'):
        raise ValueError("""if_exists parameter must be in ('fail', 'replace', 'append').""")
        
    source_pandas_df.to_sql(destination_sqlite_table, conn, if_exists=if_exists, index=index)
    print('successfully pushed dataframe to sqlite!')
    
def checksum_dataframe_with_sqlite_table_same_data_before_and_after_load(conn, source_pandas_df, destination_sqlite_table):
    """Check sums a pandas DataFrame before and after being pushed into SQLite.

    Args:
        conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
        source_pandas_df (:obj:`pd.core.frame.DataFrame`): Pandas DataFrame to compare with SQLite table.
        destination_sqlite_table (:obj:`str`): SQLite table to compare with `source_pandas_df`.
        
    Uses:
        `SQLiteQuery` class
            `run_query_return_list_of_dicts` method
        `pd.DataFrame` class
            `equals` classmethod
    Returns:
        (:obj:`bool`): Boolean `source_pandas_df` equals `destination_sqlite_table`
    """
    if type(conn) != sqlite3.Connection:
        raise TypeError('conn parameter must be type sqlite3.Connection.')    

    if type(source_pandas_df) != pd.core.frame.DataFrame  :
        raise TypeError('source_pandas_df parameter must be pd.core.frame.DataFrame.')
        
    if type(destination_sqlite_table) != str:
        raise TypeError('destination_sqlite_table parameter must be str.')
        
    check_query = "SELECT * FROM {}".format(destination_sqlite_table)
    df_check = pd.DataFrame(SQLiteQuery(conn,check_query).run_query_return_list_of_dicts())
    try:
        assert source_pandas_df.equals(df_check)
        print('dataframes equal before and after load.')
        return True
    except:
        print("dataframes not equal before and after load.")
        return False

class SQLiteQuery(object):
    def __init__(self, conn, query):
        """SQLite class used to query.
        
        Args:
            conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            query (:obj:`str`): Query to run using `sqlite3.Connection` object.
        """
        self._conn = conn
        self._query = query
        
    def run_query(self):
        """Runs self._query using self._conn.
        
        Uses:
            self._conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            self._query (:obj:`str`): Query to run using `sqlite3.Connection` object.
        """
        cursor = self._conn.cursor()
        cursor.execute(self._query)
        print('query ran successfully!')
        cursor.close()
        
    def run_query_return_list_of_dicts(self):
        """Runs self._query using self._conn returns a list_of_dicts.
        
        Uses:
            self._conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            self._query (:obj:`str`): Query to run using `sqlite3.Connection` object.
            
        Returns:
            list_of_dicts (:obj:`list` of type `dict`): List of dictionaries returned by running self._query
        """
        list_of_dicts = list()
        cursor = self._conn.cursor()
        
        for row in cursor.execute(self._query):
            d = dict(zip(next(zip(*cursor.description)), row))
            list_of_dicts.append(d)
            
        cursor.close()
        return list_of_dicts
    
    @property
    def query(self):
        """query (:obj:`str`): Query to run using `sqlite3.Connection`."""
        return self._query

    @query.setter
    def query(self, query):
        if type(query) != str:
            raise TypeError('query parameter must be str.')
        self._query = query

    @property
    def conn(self):
        """conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method."""
        return self._conn

    @conn.setter
    def conn(self, conn):
        if type(conn) != sqlite3.Connection:
            raise TypeError('conn parameter must be type sqlite3.Connection.')
        self._conn = conn

```python
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """
q1 = SQLiteQuery(conn,sql_create_projects_table)

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

q2 = SQLiteQuery(conn,sql_create_tasks_table)
```

In [19]:
def main():
    
    temp_db_path = os.path.join(os.getcwd(),'temp.db')
    
    dat = """id;name;begin_date;end_date
    1;fam training;2020-02-01;2020-02-10
    2;project1;2020-02-01;2020-03-01
    3;project2;2020-02-15;2020-04-01
    4;project3;2020-03-15;2020-05-01"""

    df = create_pandas_DataFrame_from_string(dat)
    destination_sqlite_table = 'projects'

    with create_SQLite_connection(temp_db_path) as conn:
        push_pandas_DataFrame_to_SQLite_table(conn,df,destination_sqlite_table,if_exists='replace')
        checksum_dataframe_with_sqlite_table_same_data_before_and_after_load(conn, df, destination_sqlite_table)
    
    dat2 = """id;name;priority;status_id;project_id;begin_date;end_date
    3;task3;1;0;1;2020-02-15;2020-03-01
    4;task4;1;0;1;2020-02-15;2020-03-01"""

    df2 = create_pandas_DataFrame_from_string(dat2)
    destination_sqlite_table = 'tasks'

    with create_SQLite_connection(temp_db_path) as conn:
        push_pandas_DataFrame_to_SQLite_table(conn,df2,destination_sqlite_table,if_exists='replace')
        checksum_dataframe_with_sqlite_table_same_data_before_and_after_load(conn, df2, destination_sqlite_table)
    
if __name__ == '__main__':
    main()

successfully pushed dataframe to sqlite!
dataframes equal before and after load.
successfully pushed dataframe to sqlite!
dataframes equal before and after load.
